In [1]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [2]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
train = pd.read_json("input/train.json")
#test = pd.read_json("input/test.json")

In [ ]:
only_band1 = ['band1_'+str(i) for i in range(75*75)]
only_band2 = ['band2_'+str(i) for i in range(75*75)]
both_bands = only_band1+only_band2
df = pd.concat([pd.DataFrame(test['band_1'].values.tolist(),columns = only_band1),
                pd.DataFrame(test['band_2'].values.tolist(),columns = only_band2)],axis = 1)

In [ ]:
def collect_test():
    test_data = pd.DataFrame()
    for i in range(4):
        test_piece = pd.read_csv('input/test_subparts/test'+str(i+1)+'.csv')
        test_data = pd.concat([test_data,test_piece])
        test_data = test_data.reset_index().drop(['index'],axis = 1)
    return test_data

In [ ]:
test_data = collect_test()

In [4]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [ ]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [ ]:

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=2,stratify = target_train, train_size=0.80)

### Cross Validation Folds

In [5]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches


In [6]:

def cnnmodel():
    X_input = Input(shape = (75,75,3))
    #First composite layer
    X = Conv2D(128,kernel_size = (5,5))(X_input)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Second composite layer
    X = Conv2D(256,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)
    
    #Third composite layer
    X = Conv2D(256,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Forth composite layer
    X = Conv2D(128,kernel_size = (3,3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(3,3),strides =(2,2))(X)
    X = Dropout(rate = 0.2)(X)

    #Flatten layer
    X = Flatten()(X)
    
    #First dense layer
    X = Dense(512,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Second dense layer
    X = Dense(256,activation = 'relu')(X)
    X = Dropout(rate = 0.2)(X)
   
    #Decision layer
    X = Dense(1,activation='sigmoid')(X)
    
    model = Model(inputs=X_input,outputs=X)

    return model

def model_compile(model,lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None):
   
    if weights_path:
        model.load_weights(weights_path)
    else:
        pass
    if freezing_layers:
        for layer in model.layers[:freezing_layers]:
            layer.trainable = False
    else:
        pass
    optimizer = Adam(lr = lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7, save_best_only = True):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es,msave]


In [7]:

def normal_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit(X_train_cv, y_train_cv,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result

def data_augment_fit(model,callbacks,epochs = 100,batch_size = 32,verbose = 1):
    
    result = model.fit_generator(data_augmentation(X_train_cv,y_train_cv,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=verbose,
                          validation_data=(X_valid, y_valid),
                          callbacks=callbacks)
    return result


In [ ]:
Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5', patience=100, save_best_only = False)
result = normal_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2.hdf5')
callbacks = get_callbacks(filepath = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2_stage2.hdf5', patience=30, save_best_only = True)
result = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)

In [ ]:
Cnnmodel.load_weights('model save/Model 7-2x filters Advanced CNN with DA and self TL/2018.1.2_stage2.hdf5')
Cnnmodel.evaluate(X_valid, y_valid)
#Cnnmodel.evaluate(X_train_cv, y_train_cv)

In [ ]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [ ]:
save_history(result_Cnn)

### Prediction

In [8]:
folds = list(StratifiedKFold(n_splits=5, shuffle=True,random_state = 3).split(X_train, target_train))

In [18]:
train_idx = folds[1][0]
valid_idx = folds[1][1]
X_train_cv = X_train[train_idx]
y_train_cv = target_train[train_idx]
X_valid = X_train[valid_idx]
y_valid= target_train[valid_idx]

(1282, 75, 75, 3)

In [32]:
# y_train_pred_log=0
# y_valid_pred_log = 0.0*target_train
# for j, (train_idx, valid_idx) in enumerate(folds):
#     print('\n===================FOLD=',j+1)
#     X_train_cv = X_train[train_idx]
#     y_train_cv = target_train[train_idx]
#     X_valid = X_train[valid_idx]
#     y_valid= target_train[valid_idx]
file_path_stage2 = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv1_stage1.hdf5'

In [33]:
Cnnmodel = model_compile(model = cnnmodel())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_25 (Activation)   (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_26 (Activation)   (None, 33, 33, 256)       0         
__________

In [34]:
Cnnmodel.load_weights(file_path_stage2)

In [35]:
score = Cnnmodel.evaluate(X_valid, y_valid)
print('Valid loss:', score[0])
print('Valid accuracy:', score[1])

321/321 [==============================] - 1s 3ms/step
Valid loss: 0.552208237745
Valid accuracy: 0.629283489097


In [30]:
def Model7_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 3).split(X_train, target_train))
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path_stage1 = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv'+str(j+1)+'_stage1.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = None,weights_path = None)
        callbacks = get_callbacks(filepath = file_path_stage1, patience=100, save_best_only = False)
        result1 = normal_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)
        print('Satge 1,done!')
        Cnnmodel.load_weights(filepath=file_path_stage2)
        #Getting Training Score
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        
        file_path_stage2 = 'model save/Model 7-2x filters Advanced CNN with DA and self TL/Model-7 cv'+str(j+1)+'_stage2.hdf5'
        Cnnmodel = model_compile(cnnmodel(),lr = 0.0001,decay = 0, freezing_layers = 5,weights_path = file_path_stage1)
        callbacks = get_callbacks(filepath = file_path_stage2, patience=30, save_best_only = True)
        result2 = data_augment_fit(Cnnmodel,callbacks = callbacks,epochs = 100,batch_size=32,verbose=2)
        print('Satge 2,done!')
#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        Cnnmodel.load_weights(filepath=file_path_stage2)
        #Getting Training Score
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
#         temp_test=Cnnmodel.predict(X_test)
#         y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

#     y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_valid_pred_log

In [31]:
Model7_CV(X_train,K = 5)


===================FOLD= 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 71, 71, 128)       9728      
_________________________________________________________________
activation_21 (Activation)   (None, 71, 71, 128)       0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 35, 35, 128)       0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 33, 33, 256)       295168    
_________________________________________________________________
activation_22 (Activation)   (None, 33, 33, 256)

KeyboardInterrupt: 

### Submission

In [ ]:
def Evaluation_CV(X_train,K=3):
    folds = list(StratifiedKFold(n_splits=K, shuffle=True,random_state = 2).split(X_train, target_train))
    y_test_pred_log=0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, valid_idx) in enumerate(folds):
        print('\n===================FOLD=',j+1)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_valid = X_train[valid_idx]
        y_valid= target_train[valid_idx]
        
        #Angle
#         X_angle_cv=X_angle[train_idx]
#         X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)


#         callbacks = get_callbacks(filepath=file_path, patience=5)
#         gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
#         galaxyModel= getVggAngleModel()
#         galaxyModel.fit_generator(
#                 gen_flow,
#                 steps_per_epoch=24,
#                 epochs=100,
#                 shuffle=True,
#                 verbose=1,
#                 validation_data=([X_holdout,X_angle_hold], Y_holdout),
#                 callbacks=callbacks)

        #Getting the Best Model
        #Getting Training Score
        print('cv'+str(j+1))
        score = Cnnmodel.evaluate(X_train_cv, y_train_cv)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = Cnnmodel.evaluate(X_valid, y_valid)
        print('Valid loss:', score[0])
        print('Valid accuracy:', score[1])
        print('\n')

        #Getting validation Score.
        pred_valid=Cnnmodel.predict(X_valid)
        y_valid_pred_log[valid_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=Cnnmodel.predict(X_train)
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_valid_pred_log=y_valid_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
preds = Evaluation_CV(X_train,K=5)
sub = test[['id']]
sub['is_iceberg'] = preds
sub.to_csv('5cv.csv',index = False)

### Submission(min max median stacking)

In [ ]:
def test_scores(K=5):
    y_test_pred_log=0
    test_df = test[['id']]
    for j in range(K):
        print('\n===================FOLD=',j+1)
        file_path = 'model save/Model 6-Advanced CNN with DA and self TL(cv3)/Model-6 cv'+str(j+1)+'.hdf5'
        Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
        Cnnmodel.load_weights(file_path)

        #Getting Test Scores
        temp_test=Cnnmodel.predict(X_test)
        test_df['cv'+str(j+1)]=temp_test.reshape(temp_test.shape[0])
    return test_df

In [ ]:
test_df = test_scores()

In [ ]:
def get_sub_mmm(test_df):
    prediction = test_df
    prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.9,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.1,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),np.median(prediction[prediction.columns[1:]],axis = 1)))
    sub = prediction[['id','is_iceberg']]
    sub.to_csv('sub.csv',index = False)
    return sub

In [ ]:
sub = get_sub_mmm(test_df)